# Audio Basics

A walkthough the different functions in pyramidman that handle the recording, playing and processing of audio. It is mainly for didactic and testing purposes.

In [1]:
%load_ext autoreload
%autoreload 2

from pyramidman.audio_parameters import AudioParameters
from pyramidman.basic_audio_IO import play_audio, record_audio
from pyramidman.audio_utils import get_available_microphones, get_sysdefault_microphone_index, get_all_devices_str
from pyramidman.queue_utils import record_with_queue
from pyramidman.unwrapper import unwrap

from pyramidman.Ihy import get_audio_menu_wav_file

from pyramidman.audio_utils import calibrate_microphone


import speech_recognition as sr
import numpy as np
from scipy import signal
from scipy.io import wavfile

import plotly.graph_objs as go
from IPython.display import display
import ipywidgets as widgets

supurious knowledge. So apparently speech_recoginition uses pyaudio which uses pyport to read from the strem of device. When the streem is open it will dump the recordings in a buffer from which we can read. When we read, we specify tnumber of samples (will be intenally multiplied by the size of the bytes of the sample). I do not know how big is the buffer but I know that when we read, it starts from the beginning. It is a blocking function until we have read as many samples as we specified. 

There is the following problems:
- The function to know how many samples there are to be read does not work, it just outputs the chunck size. But the data gets accumulated in the buffer from the moment we opent the stream if we do not read it. 
- Sometimes the initial 2 seconds of the stream is bullshit, it starts at the minimum for 16 bits, -32000 and the grows to 0 (its mean) over the span of 2 seconds. 

I do not know why this happens but a possible solution would be to just open the stream, sleep 2 seconds, empty the stream by reading enough samples (can be checked if the number of bytes left to read is lower than a chunck, then we know we are good. Then start as normal.

Also I do not know how to regulate the gain of the bloody microphone so that it does not overflow. It seems unlikely to be done in python, it might require specify OS calls for each case.

Also I need a way to filter the high energy noise that is not voice, maybe by fourier transform if the intensity of the requency of the 

### Instance of the class AudioParameters 

In [2]:
audio_params = AudioParameters()
unwrap(audio_params)

<AudioParameters>	object has children:
    <int>	chunk:	1024
    <int>	sample_format:	8
    <NoneType>	subtype:	None
    <int>	channels:	1
    <int>	sample_rate:	48000
    <int>	input_device_index:	0




In [3]:
audio_params.get_input_device_info()

{'name': 'HDA Intel PCH: ALC295 Analog (hw:0,0)',
 'hostapi': 0,
 'max_input_channels': 2,
 'max_output_channels': 2,
 'default_low_input_latency': 0.005804988662131519,
 'default_low_output_latency': 0.005804988662131519,
 'default_high_input_latency': 0.034829931972789115,
 'default_high_output_latency': 0.034829931972789115,
 'default_samplerate': 44100.0}

## Information of avilable devices

In [4]:
get_all_devices_str()

   0 HDA Intel PCH: ALC295 Analog (hw:0,0), ALSA (2 in, 2 out)
   1 HDA Intel PCH: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   2 HDA Intel PCH: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   3 HDA Intel PCH: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   4 HDA Intel PCH: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   5 HDA Intel PCH: HDMI 4 (hw:0,10), ALSA (0 in, 8 out)
   6 sysdefault, ALSA (128 in, 128 out)
   7 front, ALSA (0 in, 2 out)
   8 surround40, ALSA (0 in, 2 out)
   9 surround51, ALSA (0 in, 2 out)
  10 surround71, ALSA (0 in, 2 out)
  11 hdmi, ALSA (0 in, 8 out)
  12 pulse, ALSA (32 in, 32 out)
  13 dmix, ALSA (0 in, 2 out)
* 14 default, ALSA (32 in, 32 out)

In [5]:
devices_dict = get_available_microphones()
devices_dict

{'0': 'HDA Intel PCH: ALC295 Analog (hw:0,0)',
 '6': 'sysdefault',
 '12': 'pulse',
 '14': 'default'}

## Play audio

We pass it a AudioParameters instance, from it, it is going to get chunk.

In [6]:
file_to_play = "../audios/standard/english.wav"
play_audio(audio_params, filename = file_to_play )

## Recording audio

First we should select a proper mycrophone

In [7]:
audio_params.set_sysdefault_microphone_index()
audio_params.set_default_input_parameters()
unwrap(audio_params)

<AudioParameters>	object has children:
    <int>	chunk:	1024
    <int>	sample_format:	8
    <NoneType>	subtype:	None
    <int>	channels:	1
    <int>	sample_rate:	48000
    <int>	input_device_index:	6




### Call the recording function with the correct parameters

In [8]:
file_to_record = "../audios/temp/recording.wav"
record_audio(audio_params, seconds = 3, filename = file_to_record)

Recording
Finished recording


Play the recorded audio

In [9]:
play_audio(audio_params,  filename = file_to_record)

### Record using queues

It seems better, with way less in between cuts that are probably because there is less delay due to processing of the chucks. But still as time grows, this shit takes too much time.

In [10]:
filename_w = "../audios/temp/caec2.wav"
record_with_queue(audio_params, filename_w)


Recording finished: 


In [11]:
play_audio(audio_params, filename_w)

## Using a Microphone

The speech_recognition library has a Microphone class that is helpful at recording data. We can get one instance directly from the AudioParameters class.

In [12]:
mic = audio_params.get_microphone()

In [13]:
mic.device_index

6

In [14]:
mic.list_microphone_names()

['HDA Intel PCH: ALC295 Analog (hw:0,0)',
 'HDA Intel PCH: HDMI 0 (hw:0,3)',
 'HDA Intel PCH: HDMI 1 (hw:0,7)',
 'HDA Intel PCH: HDMI 2 (hw:0,8)',
 'HDA Intel PCH: HDMI 3 (hw:0,9)',
 'HDA Intel PCH: HDMI 4 (hw:0,10)',
 'sysdefault',
 'front',
 'surround40',
 'surround51',
 'surround71',
 'hdmi',
 'pulse',
 'dmix',
 'default']

We can use this instance to together with the recognizer of the library in order to capture text.

In [15]:
r = sr.Recognizer()
with mic as source:                # use the default microphone as the audio source
    audio = r.record(source, duration = 3)                   # listen for the first phrase and extract it into audio data

audio

In [16]:
unwrap(audio)

<AudioData>	object has children:
    <bytes>	frame_data
    <int>	sample_rate:	48000
    <int>	sample_width:	2

  <bytes>	frame_data has children:




In [172]:
filename_mic = '../audios/temp/hello_world.wav'

with mic as source:
    audio = r.record(source,duration = 5)

with open(filename_mic, "wb") as f:
    f.write(audio.get_wav_data())

In [175]:
play_audio(audio_params, filename_mic)

In [174]:
    # Number of bytes in the
print("SAMPLE_WIDTH: ", mic.SAMPLE_WIDTH)

SAMPLE_WIDTH:  2


### Convert audio to numpy array

In [19]:
audio_raw_data = audio.frame_data
audio_raw_data = audio.get_raw_data()
type(audio_raw_data)

bytes

In [20]:
audio_array = np.frombuffer(audio.frame_data, np.int16)
audio_array

array([-1502, -1436, -1600, ..., -4596, -4376, -4170], dtype=int16)

## Plotting of the audiowave.

In [21]:
tabs = get_audio_menu_wav_file(filename_mic)
display(tabs)

    'data': [{'line': {'color': 'deepskyblue'},
              'name': 'AAPL High'…

## Own listener implementation

Since the listener from the speech_recognition library was not good enough, we have implemented another one.

The main changes are:
- Refactoring of code.
- Removing the snowboy option
- Adding timestamp of the returned data.
- Adding 

In [22]:
from pyramidman.listener import listen

In [137]:
with mic as source:
    r.adjust_for_ambient_noise(mic, duration = 1)
    
def print_recognizer_parameters(r):
    # Maximum number of seconds of non-speaking seconds before and after the audio
    print("non_speaking_duration: ", r.non_speaking_duration)

    # Number of non-speaking seconds to be considered end of sentence.
    print("pause_threshold: ", r.pause_threshold)

    # Minimum number of seconds of a sentence.
    print("phrase_threshold: ", r.phrase_threshold)

    # The amount of energy in 
    print("energy_threshold: ", r.energy_threshold)

print_recognizer_parameters(r)
     

non_speaking_duration:  0.5
pause_threshold:  0.8
phrase_threshold:  0.3
energy_threshold:  1614.8541607580403


In [177]:
import time
r.non_speaking_duration = 0.2
r.pause_threshold = 1.2
r.energy_threshold = 3000
with mic as source:
    time.sleep(1)
    audio = listen(r, source, timeout = 1, phrase_time_limit=6)

Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  2048
Samples to read: 4800 Bytes read:  9600 Available bits:  56


WaitTimeoutError: listening timed out while waiting for phrase to start

In [169]:
with open(filename_mic, "wb") as f:
    f.write(audio.get_wav_data())

In [170]:
tabs = get_audio_menu_wav_file(filename_mic)
display(tabs)

    'data': [{'line': {'color': 'deepskyblue'},
              'name': 'AAPL High'…